In [1]:
# Kaggle API 설정
!pip install -q kaggle

# Kaggle API 키 업로드 (kaggle.json 파일 필요)
from google.colab import files
files.upload()  # kaggle.json 업로드

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드
!kaggle datasets download -d andrewmvd/road-sign-detection
!unzip -q road-sign-detection.zip -d /content/road_sign_data

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/road-sign-detection
License(s): CC0-1.0
 70% 153M/218M [00:00<00:00, 1.60GB/s]
100% 218M/218M [00:00<00:00, 1.01GB/s]


In [2]:
import os
import shutil
from pathlib import Path

# 현재 구조 확인
!ls -la /content/road_sign_data/

total 76
drwxr-xr-x 4 root root  4096 Jan  7 18:34 .
drwxr-xr-x 1 root root  4096 Jan  7 18:34 ..
drwxr-xr-x 2 root root 32768 Jan  7 18:34 annotations
drwxr-xr-x 2 root root 32768 Jan  7 18:34 images


In [3]:
%cd /content/road_sign_data

/content/road_sign_data


In [4]:
!git clone https://github.com/ssaru/convert2Yolo.git

Cloning into 'convert2Yolo'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 215 (delta 38), reused 35 (delta 35), pack-reused 172 (from 1)
Receiving objects: 100% (215/215), 994.67 KiB | 3.62 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [5]:
%cd convert2Yolo/

/content/road_sign_data/convert2Yolo


In [6]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 12.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [11]:
!python example.py \
    --datasets VOC \
    --img_path /content/road_sign_data/images \
    --label /content/road_sign_data/annotations \
    --convert_output_path /content/road_sign_data/converted_yolo_labels \
    --img_type '.png' \
    --manifest_path /content/road_sign_data \
    --cls_list_file ./voc.names


VOC Parsing:   |████████████████████████████████████████| 100.0% (877/877)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (877/877)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (877/877)  Complete



In [8]:
# 경로 설정
base_path = Path('/content/road_sign_data')
output_path = Path('/content/yolo_datasets')

# YOLO 디렉토리 구조 생성
# 이미지, 라벨을 각각 학습용/테스트용으로 나눈 뒤 저장할 폴더 생성
(output_path / 'images' / 'train').mkdir(parents=True, exist_ok=True)
(output_path / 'images' / 'test').mkdir(parents=True, exist_ok=True)
(output_path / 'labels' / 'train').mkdir(parents=True, exist_ok=True)
(output_path / 'labels' / 'test').mkdir(parents=True, exist_ok=True)

In [9]:
# kaggle에 기재된 4개의 클래스명
classes = ['trafficlight', 'stop', 'speedlimit', 'crosswalk']

In [12]:
image_files = list(base_path.glob('images/*.png'))
# print(image_files[:5])

annotaion_files = list(base_path.glob('converted_yolo_labels/*.txt'))

print(len(image_files), len(annotaion_files))

877 877


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_images, test_images, train_annots, test_annots = train_test_split(
    image_files, annotaion_files, test_size=0.2, random_state=42
)

print(len(train_images), len(test_images))

701 176


In [15]:
# Train Data
print('Processing training data..')

for img_path, txt_path in zip(train_images, train_annots):
  # 이미지 복사
  shutil.copy(img_path, output_path / 'images' / 'train' / img_path.name)
  shutil.copy(txt_path, output_path / 'labels' / 'train' / txt_path.name)
print('완료!')

Processing training data..
완료!


In [16]:
# Validation Data
print('Processing val data..')

for img, txt in zip(test_images, test_annots):
  shutil.copy(img, output_path / 'images' / 'test' / img.name)
  shutil.copy(img, output_path / 'labels' / 'test' / txt.name)
print('완료!')

Processing val data..
완료!


In [23]:
import yaml

In [27]:
# yaml 파일 작성
data_yaml = {
    'path': '/content/yolo_datasets',
    'train': 'images/train',
    'val': 'images/val',
    'nc': len(classes),
    'names': classes
}

In [33]:
!ls /content/yolo_datasets/labels/val | head


road104.txt
road106.txt
road110.txt
road112.txt
road113.txt
road116.txt
road11.txt
road126.txt
road135.txt
road138.txt


In [28]:
with open('/content/yolo_datasets/data.yaml', 'w') as f:
  yaml.dump(data_yaml, f, sort_keys=False)

# 확인
!cat /content/yolo_datasets/data.yaml

path: /content/yolo_datasets
train: images/train
val: images/val
nc: 4
names:
- trafficlight
- stop
- speedlimit
- crosswalk


In [20]:
# YOLOv8 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00


In [21]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [35]:
model = YOLO('yolov8n.pt')

# 학습
results = model.train(
  data='/content/yolo_datasets/data.yaml',
  epochs=20,
  batch=32,
  imgsz=640,
  device=0,
  workers=2, # 2개의 쓰레드
  name='train_result'
)

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_datasets/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_result2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tru

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:836: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/20      4.83G          0      217.5          0          0        640: 100% ━━━━━━━━━━━━ 5/5 2.4it/s 2.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.9it/s 1.0s
                   all        140          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:836: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/20      4.83G          0      213.3          0          0        640: 100% ━━━━━━━━━━━━ 5/5 2.2it/s 2.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 3.0it/s 1.0s
                   all        140          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      20/20      4.83G          0      147.2          0          0        640: 100% ━━━━━━━━━━━━ 5/5 3.5it/s 1.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 3.5it/s 0.9s
                   all        140          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:836: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



20 epochs completed in 0.025 hours.
Optimizer stripped from /content/yolo_datasets/runs/detect/train_result2/weights/last.pt, 6.2MB
Optimizer stripped from /content/yolo_datasets/runs/detect/train_result2/weights/best.pt, 6.2MB

Validating /content/yolo_datasets/runs/detect/train_result2/weights/best.pt...
Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.8it/s 1.1s


/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:654: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:700: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:700: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all        140          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to /content/yolo_datasets/runs/detect/train_result2


In [36]:
!ls /content/yolo_datasets/runs/detect/train_result2

args.yaml			 results.png		val_batch0_pred.jpg
BoxF1_curve.png			 train_batch0.jpg	val_batch1_labels.jpg
BoxP_curve.png			 train_batch1.jpg	val_batch1_pred.jpg
BoxPR_curve.png			 train_batch2.jpg	val_batch2_labels.jpg
BoxR_curve.png			 train_batch50.jpg	val_batch2_pred.jpg
confusion_matrix_normalized.png  train_batch51.jpg	weights
confusion_matrix.png		 train_batch52.jpg
results.csv			 val_batch0_labels.jpg


In [37]:
# 학습된 모델로 검증을 해보자

model = YOLO('/content/yolo_datasets/runs/detect/train_result/weights/best.pt')

results = model.val(
    source='/content/yolo_datasets/data.yaml',
    imgsz=640,
    iou=0.5,
    device=0,
    workers=2,               # 데이터 로드 시 병렬 처리할 워커 수
    half=True,               # FP16 연산 활성화 (속도 향상)
    split="val"
)

print(results)

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2344.5±1424.3 MB/s, size: 182.3 KB)
val: Scanning /content/yolo_datasets/labels/val.cache... 36 images, 140 backgrounds, 36 corrupt: 100% ━━━━━━━━━━━━ 176/176 354.6Kit/s 0.0s
val: /content/yolo_datasets/images/val/road11.png: ignoring corrupt image/label: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
val: /content/yolo_datasets/images/val/road110.png: ignoring corrupt image/label: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
val: /content/yolo_datasets/images/val/road138.png: ignoring corrupt image/label: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
val: /content/yolo_datasets/images/val/road200.png: ignoring corrupt image/label: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
v

/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:654: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:700: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultralytics/utils/metrics.py:700: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.12/dist-packages/ultraly

                   all        140          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, cannot compute metrics without labels
Speed: 2.3ms preprocess, 14.3ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to /content/yolo_datasets/runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([], dtype=int64)
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7afc8b4e1970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,

In [40]:
model = YOLO('/content/yolo_datasets/runs/detect/train_result/weights/best.pt')

results = model.predict(
    source='/content/yolo_datasets/images/val',
    imgsz=640,
    conf=0.05,
    device=0,
    save=True,
    save_txt=True,
    save_conf=True
)
print(results)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
       [[124, 130, 116],
        [127, 131, 119],
        [128, 132, 120],
        ...,
        [157, 165, 154],
        [160, 168, 155],
        [162, 171, 158]],

       [[125, 131, 117],
        [126, 132, 118],
        [125, 131, 117],
        ...,
        [152, 163, 148],
        [137, 148, 132],
        [135, 146, 130]]], dtype=uint8)
orig_shape: (400, 300)
path: '/content/yolo_datasets/images/val/road467.png'
probs: None
save_dir: '/content/yolo_datasets/runs/detect/predict'
speed: {'preprocess': 2.4052320000009786, 'inference': 7.4639669999214675, 'postprocess': 0.6951649997972709}, ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'trafficlight', 1: 'stop', 2: 'speedlimit', 3: 'crosswalk'}
obb: None
orig_img: array([[[237, 202, 126],
        [239, 201, 128],
        [238, 200, 127],
        ...,
        [243, 205, 131],
        [242, 205, 129],
  